In [1]:
import pathlib
import os
import zipfile


from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SparkSession
import tqdm

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.12.0 pyspark-shell'

In [20]:
! spark-submit --version

23/07/21 17:10:56 WARN Utils: Your hostname, asus-notebook resolves to a loopback address: 127.0.1.1; using 192.168.0.4 instead (on interface wlp3s0)
23/07/21 17:10:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.1
      /_/
                        
Using Scala version 2.12.17, OpenJDK 64-Bit Server VM, 11.0.17
Branch HEAD
Compiled by user centos on 2023-06-19T23:01:01Z
Revision 6b1ff22dde1ead51cbf370be6e48a802daae58b6
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
conf = SparkConf()
conf.setMaster("local[*]").setAppName("Tax Service Opendata Processor")

In [4]:
conf.get("spark.master")

'local[*]'

In [5]:
sc = SparkContext(conf=conf)

23/07/25 17:44:09 WARN Utils: Your hostname, asus-notebook resolves to a loopback address: 127.0.1.1; using 192.168.1.186 instead (on interface wlp3s0)
23/07/25 17:44:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/pavel/.venv/hse/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pavel/.ivy2/cache
The jars for the packages stored in: /home/pavel/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4b7acf38-23bb-421a-9e05-c770acc4af1b;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.12.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.3 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 475ms :: artifacts dl 21ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.12.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| se

In [6]:
sc.uiWebUrl

'http://192.168.1.186:4040'

In [7]:
session = SparkSession(sc)

In [11]:
files = (
    "debtam/VO_OTKRDAN6_9965_9965_20221220_0a0a9fa4-008f-4835-aee7-c6281ec6d84d.xml",
    "debtam/VO_OTKRDAN6_9965_9965_20221220_0a0cb2e9-a1d4-4e59-b8fa-e07abe335a48.xml",
    "debtam/VO_OTKRDAN6_9965_9965_20221220_0a0d0116-6812-4d72-9359-ad9fadc7e296.xml"
)
df1 = session.read.format("xml").options(rowTag="Документ").load(",".join(files))
df1.printSchema()

root
 |-- _ДатаДок: string (nullable = true)
 |-- _ДатаСост: string (nullable = true)
 |-- _ИдДок: string (nullable = true)
 |-- СведНП: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _ИННЮЛ: long (nullable = true)
 |    |-- _НаимОрг: string (nullable = true)
 |-- СведНедоим: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _НаимНалог: string (nullable = true)
 |    |    |-- _ОбщСумНедоим: double (nullable = true)
 |    |    |-- _СумНедНалог: double (nullable = true)
 |    |    |-- _СумПени: double (nullable = true)
 |    |    |-- _СумШтраф: double (nullable = true)



In [12]:
df1.count()

222

In [20]:
df2 = session.read.format("xml").options(rowTag="Документ").load("paytax/VO_OTKRDAN4_9965_9965_20221129_0a0bc4ef-96aa-4fb7-835e-f5991b43c51a.xml")
df2.printSchema()

root
 |-- _ДатаДок: string (nullable = true)
 |-- _ДатаСост: string (nullable = true)
 |-- _ИдДок: string (nullable = true)
 |-- СвУплСумНал: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _НаимНалог: string (nullable = true)
 |    |    |-- _СумУплНал: double (nullable = true)
 |-- СведНП: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _ИННЮЛ: long (nullable = true)
 |    |-- _НаимОрг: string (nullable = true)



In [21]:
df3 = session.read.format("xml").options(rowTag="Документ").load("revexp/VO_OTKRDAN_5_9965_9965_20230626_0a24217e-8d61-4d2a-8487-ffa1fb686028.xml")
df3.printSchema()

root
 |-- _ДатаДок: string (nullable = true)
 |-- _ДатаСост: string (nullable = true)
 |-- _ИдДок: string (nullable = true)
 |-- СведДохРасх: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _СумДоход: double (nullable = true)
 |    |-- _СумРасход: double (nullable = true)
 |-- СведНП: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _ИННЮЛ: long (nullable = true)
 |    |-- _НаимОрг: string (nullable = true)



In [18]:
df4 = session.read.format("xml").options(rowTag="Документ").load("rmsp/VO_RRMSPSV_0000_9965_20170410_0a1a5aa4-6a6d-4f93-91b6-ead7ba8f83c3.xml")
df4.printSchema()

root
 |-- _ВидСубМСП: long (nullable = true)
 |-- _ДатаВклМСП: string (nullable = true)
 |-- _ДатаСост: string (nullable = true)
 |-- _ИдДок: string (nullable = true)
 |-- _КатСубМСП: long (nullable = true)
 |-- _ПризНовМСП: long (nullable = true)
 |-- ИПВклМСП: struct (nullable = true)
 |    |-- _ИННФЛ: long (nullable = true)
 |    |-- ФИОИП: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _Имя: string (nullable = true)
 |    |    |-- _Отчество: string (nullable = true)
 |    |    |-- _Фамилия: string (nullable = true)
 |-- ОргВклМСП: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _ИННЮЛ: long (nullable = true)
 |    |-- _НаимОрг: string (nullable = true)
 |    |-- _НаимОргСокр: string (nullable = true)
 |-- СвЛиценз: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _ДатаКонЛиценз: string (nullable = true)
 |    |    |-- _ДатаЛиценз: string (nullable = true)
 |    |    |-- _Дат

In [16]:
df5 = session.read.format("xml").options(rowTag="Документ").load("rsmppp/VO_SVMSP_0000_9965_20230715_0a1d08a0-73d7-4979-a7b1-08f7d518056b.xml")
df5.printSchema()

root
 |-- _ДатаСост: string (nullable = true)
 |-- _ИдДок: string (nullable = true)
 |-- СвПредПод: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _ВидПП: long (nullable = true)
 |    |    |-- _ДатаОбнов: string (nullable = true)
 |    |    |-- _ДатаПрекр: string (nullable = true)
 |    |    |-- _ДатаПрин: string (nullable = true)
 |    |    |-- _ДатаСвед: string (nullable = true)
 |    |    |-- _ИННЮЛ: long (nullable = true)
 |    |    |-- _ИнфНаруш: long (nullable = true)
 |    |    |-- _КатСуб: long (nullable = true)
 |    |    |-- _НаимОрг: string (nullable = true)
 |    |    |-- _НомерПод: long (nullable = true)
 |    |    |-- _СрокПод: string (nullable = true)
 |    |    |-- ВидПод: struct (nullable = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _КодВид: long (nullable = true)
 |    |    |    |-- _НаимВид: string (nullable = true)
 |    |    |-- Нарушения: struct (nullable = true)
 |    |    |    |-- _VALUE

In [17]:
df5.select("СвФЛ._ИННФЛ").show()

+------------+
|      _ИННФЛ|
+------------+
|890304244382|
|890304245019|
|890304245330|
|890304246929|
|890304247591|
|890304248154|
|890304251679|
|890304252094|
|890304252947|
|890304257060|
|890304261211|
|890304261780|
|890304262800|
|890304269186|
|890304277109|
|890304288598|
|890304309390|
|890304318517|
|890304356488|
|890304359129|
+------------+
only showing top 20 rows



In [8]:
df6 = session.read.format("xml").options(rowTag="Документ").load("sshr/VO_OTKRDAN_3_9965_9965_20230625_0a7bc178-6202-428a-9546-80b9cc00f7dc.xml")
df6.printSchema()

In [15]:
df6.select("СведНП._ИННЮЛ").count()

900

In [13]:
import os
import zipfile

In [14]:
PATH = "rmsp/data-01102019-structure-08012016.zip"

In [15]:
parts = []
with zipfile.ZipFile(PATH) as archive:
    for xml_file in archive.infolist():
        if ".xml" not in xml_file.filename:
            continue
        content = archive.read(xml_file).decode("utf8")
        
        path = archive.extract(xml_file, path="/tmp")
        print(path)
        #df = session.read.format("xml").options(rowTag="Документ").load(path)
        #parts.append(df)
        os.remove(path)       

/tmp/VO_RRMSPSV_0000_9965_20190110_00019b6b-66db-47a3-9dc1-531f91eed871.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_00097c83-44ac-448c-826b-86b249e212fd.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_0015eee3-3cf5-408d-97d0-3e03f2a7d29d.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_001691bf-a015-49b5-b074-b5088f2a4033.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_0018d482-6717-4293-b57c-ac46e5fe7167.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_001ce934-a384-4250-ae70-5a3da6ecf159.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_001cf025-d3ca-4847-af64-8e5fe228c7b0.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_00230eda-e4a9-417d-9b30-3ddcfb586758.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_00343daa-c78a-491a-a7fe-ac584f6a8ae1.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_003bb278-bc3f-4cd6-8f0c-8054bba7e4e7.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_003c43e1-d2b4-436c-8426-8a8ca550ee57.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_00412def-0e24-4b50-84af-889b97d2b8c1.xml
/tmp/VO_RRMSPSV_0000_9965_20190110_0049731a-6d6d-4ae6-b4f3-4e56fb221755.xml
/tmp/VO_RRMS

KeyboardInterrupt: 

In [29]:
from functools import reduce

In [30]:
from pyspark.sql import DataFrame

In [31]:
data = reduce(DataFrame.union, parts)

AnalysisException: [NUM_COLUMNS_MISMATCH] UNION can only be performed on inputs with the same number of columns, but the first input has 11 columns and the 5th input has 10 columns.;
'Union false, false
:- Relation [_ВидСубМСП#4960L,_ДатаВклМСП#4961,_ДатаСост#4962,_ИдДок#4963,_КатСубМСП#4964L,_ПризНовМСП#4965L,ИПВклМСП#4966,ОргВклМСП#4967,СвЛиценз#4968,СвОКВЭД#4969,СведМН#4970] XmlRelation(com.databricks.spark.xml.DefaultSource$$Lambda$1163/0x00000008409b8040@69da3405,Some(/tmp/VO_RRMSPSV_0000_9965_20190110_00019b6b-66db-47a3-9dc1-531f91eed871.xml),Map(rowtag -> Документ, path -> /tmp/VO_RRMSPSV_0000_9965_20190110_00019b6b-66db-47a3-9dc1-531f91eed871.xml),null)
:- Relation [_ВидСубМСП#4982L,_ДатаВклМСП#4983,_ДатаСост#4984,_ИдДок#4985,_КатСубМСП#4986L,_ПризНовМСП#4987L,ИПВклМСП#4988,ОргВклМСП#4989,СвЛиценз#4990,СвОКВЭД#4991,СведМН#4992] XmlRelation(com.databricks.spark.xml.DefaultSource$$Lambda$1163/0x00000008409b8040@e726d84,Some(/tmp/VO_RRMSPSV_0000_9965_20190110_00097c83-44ac-448c-82

In [32]:
parts[0].printSchema()

root
 |-- _ВидСубМСП: long (nullable = true)
 |-- _ДатаВклМСП: string (nullable = true)
 |-- _ДатаСост: string (nullable = true)
 |-- _ИдДок: string (nullable = true)
 |-- _КатСубМСП: long (nullable = true)
 |-- _ПризНовМСП: long (nullable = true)
 |-- ИПВклМСП: struct (nullable = true)
 |    |-- _ИННФЛ: long (nullable = true)
 |    |-- ФИОИП: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _Имя: string (nullable = true)
 |    |    |-- _Отчество: string (nullable = true)
 |    |    |-- _Фамилия: string (nullable = true)
 |-- ОргВклМСП: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _ИННЮЛ: long (nullable = true)
 |    |-- _НаимОрг: string (nullable = true)
 |    |-- _НаимОргСокр: string (nullable = true)
 |-- СвЛиценз: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _ДатаКонЛиценз: string (nullable = true)
 |    |    |-- _ДатаЛиценз: string (nullable = true)
 |    |    |-- _Дат

In [33]:
parts[4].printSchema()

root
 |-- _ВидСубМСП: long (nullable = true)
 |-- _ДатаВклМСП: string (nullable = true)
 |-- _ДатаСост: string (nullable = true)
 |-- _ИдДок: string (nullable = true)
 |-- _КатСубМСП: long (nullable = true)
 |-- _ПризНовМСП: long (nullable = true)
 |-- ОргВклМСП: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _ИННЮЛ: long (nullable = true)
 |    |-- _НаимОрг: string (nullable = true)
 |    |-- _НаимОргСокр: string (nullable = true)
 |-- СвЛиценз: struct (nullable = true)
 |    |-- _ДатаКонЛиценз: string (nullable = true)
 |    |-- _ДатаЛиценз: string (nullable = true)
 |    |-- _ДатаНачЛиценз: string (nullable = true)
 |    |-- _НомЛиценз: string (nullable = true)
 |    |-- НаимЛицВД: string (nullable = true)
 |    |-- СведАдрЛицВД: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- СвОКВЭД: struct (nullable = true)
 |    |-- СвОКВЭДДоп: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    

In [8]:
import pathlib

In [10]:
path = pathlib.Path("/home/pavel/paytax/")
[str(f) for f in path.iterdir()]

['/home/pavel/paytax/VO_OTKRDAN4_9965_9965_20221129_609aa035-75b0-40db-a182-5a6e400aea89.xml',
 '/home/pavel/paytax/VO_OTKRDAN4_9965_9965_20221129_c01f611b-b885-4413-b888-142e160ddbaa.xml',
 '/home/pavel/paytax/VO_OTKRDAN4_9965_9965_20221129_ced33bee-cc77-4e20-9f55-18e34bd4da74.xml',
 '/home/pavel/paytax/VO_OTKRDAN4_9965_9965_20221129_f7f21517-69d0-46fc-bb42-c2ef0753a4bb.xml',
 '/home/pavel/paytax/VO_OTKRDAN4_9965_9965_20221129_80004acb-c9f7-4a7e-abc7-bd279e43fb2e.xml',
 '/home/pavel/paytax/VO_OTKRDAN4_9965_9965_20221129_f5f951af-701a-4cfb-a311-b0dae85c4e50.xml',
 '/home/pavel/paytax/VO_OTKRDAN4_9965_9965_20221129_99bf5b99-6d1b-4e53-84ef-cbf85353c0b1.xml',
 '/home/pavel/paytax/VO_OTKRDAN4_9965_9965_20221129_30bd4062-9281-48d8-8ce4-d5dca72a1c6f.xml',
 '/home/pavel/paytax/VO_OTKRDAN4_9965_9965_20221129_ab859555-9036-4c83-9339-9b1f9a6cc5b4.xml',
 '/home/pavel/paytax/VO_OTKRDAN4_9965_9965_20221129_4c057187-87af-4182-a5c5-3a9b22abf602.xml',
 '/home/pavel/paytax/VO_OTKRDAN4_9965_9965_2022112

In [15]:
df = session.read.format("xml").options(rowTag="Документ")\
    .load(",".join([str(f) for f in path.iterdir()][:1260]))

In [16]:
df.write.parquet("/home/pavel/paytax.parquet")

In [13]:
df2 = session.read.parquet("/home/pavel/paytax.parquet/")

In [30]:
df.printSchema()

root
 |-- _ДатаДок: string (nullable = true)
 |-- _ДатаСост: string (nullable = true)
 |-- _ИдДок: string (nullable = true)
 |-- СвУплСумНал: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _НаимНалог: string (nullable = true)
 |    |    |-- _СумУплНал: double (nullable = true)
 |-- СведНП: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _ИННЮЛ: long (nullable = true)
 |    |-- _НаимОрг: string (nullable = true)



In [29]:
df.select("СвУплСумНал._СумУплНал").show(20)

+--------------------+
|          _СумУплНал|
+--------------------+
|[0.0, 0.0, 0.0, 0...|
|[0.0, 263000.0, 0...|
|[22078.0, 13737.6...|
|[119115.0, 75827....|
|[0.0, 0.0, 0.0, 0...|
|[199931.0, 15819....|
|[11073.51, 37832....|
|[0.0, 0.0, 0.0, 0...|
|[0.0, 0.0, 0.0, 0...|
|[0.0, 0.0, 0.0, 0...|
|[62697.0, 124968....|
|[43191.0, 148619....|
|[0.0, 0.0, 0.0, 0...|
|[0.0, 5433501.64,...|
|[218348.15, 28782...|
|[0.0, 0.0, 0.0, 0...|
|[0.0, 0.0, 0.0, 0...|
|[0.0, 0.0, 0.0, 2...|
|[252790.0, 4784.0...|
|[355572.3, 104985...|
+--------------------+
only showing top 20 rows



In [8]:
sc.addFile("debtam/data-20221201-structure-20181201.zip")

In [10]:
sc.listFiles

['file:///home/pavel/.ivy2/jars/com.databricks_spark-xml_2.12-0.12.0.jar',
 'file:///home/pavel/.ivy2/jars/org.glassfish.jaxb_txw2-2.3.3.jar',
 'file:///home/pavel/.ivy2/jars/org.apache.ws.xmlschema_xmlschema-core-2.2.5.jar',
 'file:///home/pavel/.ivy2/jars/commons-io_commons-io-2.8.0.jar',
 'file:/media/data/Павел/Документы/Учёба/Аспирантура/tax-service-opendata/debtam/data-20221201-structure-20181201.zip']

In [8]:
with zipfile.ZipFile("debtam/data-20221201-structure-20181201.zip") as archive:
    for fn in tqdm.tqdm(archive.infolist()[:126]):
        if "xml" not in fn.filename:
            continue
        extracted = archive.extract(fn, path="/tmp")
        sc.addFile(extracted)
        os.remove(extracted)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 126/126 [00:01<00:00, 81.65it/s]


In [9]:
sc.listFiles

['file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_009bb515-50d0-404a-8af6-5feb51e3ff3f.xml',
 'file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_0018d159-3b33-4926-956e-c9477967db55.xml',
 'file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_02807281-e7d3-4e81-b247-54b0a38e3999.xml',
 'file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_02971793-80f4-44b4-9424-938ee5f2c624.xml',
 'file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_023b3a37-1f1a-4f76-b42b-2ed22ee13291.xml',
 'file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_028871d4-d8ff-486c-96bc-5ff978b3422c.xml',
 'file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_01790ddc-9454-4ec2-98c5-fc0144aa972f.xml',
 'file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_01a49efd-27e9-48c2-abd9-d8e5944d5ac3.xml',
 'file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_014ea047-5668-4e40-a4f6-3a0ac77b3e87.xml',
 'file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_00f2fbce-f4c1-4940-be0a-3a19cc3c6f6e.xml',
 'file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_0120bc53-a24c-465e-b

In [10]:
df = session.read.format("xml").options(rowTag="Документ")\
    .load(",".join([SparkFiles.get(pathlib.Path(f).name) for f in sc.listFiles[:1] if "xml" in f]))

23/07/25 17:44:43 ERROR Executor: Exception in task 0.0 in stage 0.0 (TID 0)
org.apache.spark.SparkException: File /tmp/spark-9dc5f5d6-53cc-449b-99b4-e77dcba0d2e6/userFiles-eaccff10-6c06-4d34-aa5d-e8225486d5e8/VO_OTKRDAN6_9965_9965_20221220_022feae4-262b-4fa4-a98e-2e782fd423eb.xml exists and does not match contents of file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_022feae4-262b-4fa4-a98e-2e782fd423eb.xml
	at org.apache.spark.util.Utils$.copyFile(Utils.scala:720)
	at org.apache.spark.util.Utils$.doFetchFile(Utils.scala:813)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:555)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1033)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1030)
	at scala.collection.TraversableLike$WithFilter.$anonfun$foreach$1(TraversableLike.scala:985)
	at scala.collection.mutable.HashMap.$anonfun$foreach$1(HashMap.scala:149)
	at scala.collection.mutable.HashTable.foreachEntry(H

Py4JJavaError: An error occurred while calling o163.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (192.168.1.186 executor driver): org.apache.spark.SparkException: File /tmp/spark-9dc5f5d6-53cc-449b-99b4-e77dcba0d2e6/userFiles-eaccff10-6c06-4d34-aa5d-e8225486d5e8/VO_OTKRDAN6_9965_9965_20221220_022feae4-262b-4fa4-a98e-2e782fd423eb.xml exists and does not match contents of file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_022feae4-262b-4fa4-a98e-2e782fd423eb.xml
	at org.apache.spark.util.Utils$.copyFile(Utils.scala:720)
	at org.apache.spark.util.Utils$.doFetchFile(Utils.scala:813)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:555)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1033)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1030)
	at scala.collection.TraversableLike$WithFilter

In [12]:
session.read.format("xml").options(rowTag="Документ")\
    .load(SparkFiles.get(pathlib.Path(sc.listFiles[0]).name))

23/07/25 17:45:05 ERROR Executor: Exception in task 0.0 in stage 1.0 (TID 1)
org.apache.spark.SparkException: File /tmp/spark-9dc5f5d6-53cc-449b-99b4-e77dcba0d2e6/userFiles-eaccff10-6c06-4d34-aa5d-e8225486d5e8/VO_OTKRDAN6_9965_9965_20221220_022feae4-262b-4fa4-a98e-2e782fd423eb.xml exists and does not match contents of file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_022feae4-262b-4fa4-a98e-2e782fd423eb.xml
	at org.apache.spark.util.Utils$.copyFile(Utils.scala:720)
	at org.apache.spark.util.Utils$.doFetchFile(Utils.scala:813)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:555)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1033)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1030)
	at scala.collection.TraversableLike$WithFilter.$anonfun$foreach$1(TraversableLike.scala:985)
	at scala.collection.mutable.HashMap.$anonfun$foreach$1(HashMap.scala:149)
	at scala.collection.mutable.HashTable.foreachEntry(H

Py4JJavaError: An error occurred while calling o183.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (192.168.1.186 executor driver): org.apache.spark.SparkException: File /tmp/spark-9dc5f5d6-53cc-449b-99b4-e77dcba0d2e6/userFiles-eaccff10-6c06-4d34-aa5d-e8225486d5e8/VO_OTKRDAN6_9965_9965_20221220_022feae4-262b-4fa4-a98e-2e782fd423eb.xml exists and does not match contents of file:/tmp/VO_OT6/VO_OTKRDAN6_9965_9965_20221220_022feae4-262b-4fa4-a98e-2e782fd423eb.xml
	at org.apache.spark.util.Utils$.copyFile(Utils.scala:720)
	at org.apache.spark.util.Utils$.doFetchFile(Utils.scala:813)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:555)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5(Executor.scala:1033)
	at org.apache.spark.executor.Executor.$anonfun$updateDependencies$5$adapted(Executor.scala:1030)
	at scala.collection.TraversableLike$WithFilter

In [37]:
[SparkFiles.get(pathlib.Path(f).name) for f in sc.listFiles[:10] if "xml" in f]

['/tmp/spark-94ce11fd-923b-419b-9485-de25a92e3568/userFiles-c11edb4e-80f4-4351-b187-f107f8452cf0/VO_OTKRDAN6_9965_9965_20221220_0735c21d-9989-42c5-be62-3f43bf7d9845.xml',
 '/tmp/spark-94ce11fd-923b-419b-9485-de25a92e3568/userFiles-c11edb4e-80f4-4351-b187-f107f8452cf0/VO_OTKRDAN6_9965_9965_20221220_28e969e7-6b3c-4933-836a-bef943e8ad73.xml',
 '/tmp/spark-94ce11fd-923b-419b-9485-de25a92e3568/userFiles-c11edb4e-80f4-4351-b187-f107f8452cf0/VO_OTKRDAN6_9965_9965_20221220_81f0aa7d-dc8a-49de-bec4-626b1cf88bfa.xml',
 '/tmp/spark-94ce11fd-923b-419b-9485-de25a92e3568/userFiles-c11edb4e-80f4-4351-b187-f107f8452cf0/VO_OTKRDAN6_9965_9965_20221220_cbb11f00-2eb8-4dbf-bf13-4e0bb91b5c0b.xml',
 '/tmp/spark-94ce11fd-923b-419b-9485-de25a92e3568/userFiles-c11edb4e-80f4-4351-b187-f107f8452cf0/VO_OTKRDAN6_9965_9965_20221220_f12c922c-3a54-4443-8675-cbf3219d4626.xml',
 '/tmp/spark-94ce11fd-923b-419b-9485-de25a92e3568/userFiles-c11edb4e-80f4-4351-b187-f107f8452cf0/VO_OTKRDAN6_9965_9965_20221220_0da0a205-f697-400